In [1]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# ML
from regression import SparseKRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from errors import MAE, RMSE

# Utilities
import h5py
import json
from tqdm.notebook import tqdm
from project_utils import load_structures_from_hdf5

In [2]:
# Load train and test sets
train_idxs = np.loadtxt('../Processed_Data/DEEM_10k/train.idxs', dtype=int)
test_idxs = np.loadtxt('../Processed_Data/DEEM_10k/test.idxs', dtype=int)

# Total number of structures
n_structures = train_idxs.size + test_idxs.size

In [3]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [4]:
# Cross validation splitting (relative to train indices)
k = 5
cv_idxs = np.arange(0, train_idxs.size)
np.random.shuffle(cv_idxs)
try:
    cv_idxs = np.split(cv_idxs, k)
    np.savetxt('../Processed_Data/DEEM_10k/cv.idxs', np.stack(cv_idxs, axis=1), fmt='%d')
except ValueError:
    print('Error: number of points in the training set must be divisible by the number of folds')

In [5]:
# Set ranges of kernel gamma and regularization (sigma and reg)
kernel_type = 'gaussian'
gamma = np.logspace(-3, 2, 6) #np.logspace(-3, 2, 11)
sigma = np.logspace(-3, 2, 6) #np.logspace(-3, 2, 11)
reg_type = 'scalar'
reg = np.logspace(-4, 1, 6) #np.logspace(-4, 1, 11)

In [6]:
# Make directory in which to store models
if not os.path.exists('../Processed_Data/Models'):
    os.mkdir('../Processed_Data/Models')

In [7]:
# Initialize datatype list for storing errors
dt_list = [('gamma', 'f8'), 
           ('sigma', 'f8'), 
           ('reg', 'f8'), 
           ('mae_train', 'f8', (k,)),
           ('mae_validate', 'f8', (k,)),
           ('rmse_train', 'f8', (k,)),
           ('rmse_validate', 'f8', (k,))]

# Save the data type for future reference
with open('../Processed_Data/Models/regression_optimization_dtype.json', 'w') as f:
    json.dump(dt_list, f)

# Make numpy data type from the list
dt = np.dtype(dt_list)

# DEEM_10k

In [8]:
# Set property names for loading
property_names = ['volumes', 'energies']

# Load structure properties
structure_properties = {}
for pn in property_names:
    structure_properties[pn] = np.loadtxt(f'../Processed_Data/DEEM_10k/Data/structure_{pn}.dat')

In [9]:
# Set structure labels for loading from the HDF5 file
n_digits = len(str(n_structures - 1))
datasets = [str(i).zfill(n_digits) for i in train_idxs]

In [ ]:
# Optimize hyperparameters

# Loop over cutoffs
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=False):
    
    # Set data directory
    data_dir = f'../Processed_Data/DEEM_10k/Data/{cutoff}'
    
    # Set working directory
    work_dir = f'../Processed_Data/Models/{cutoff}'
    if not os.path.exists(work_dir):
        os.mkdir(work_dir)
    
    # Read SOAPs in training set
    soaps = load_structures_from_hdf5(f'{data_dir}/soaps.hdf5',
                                      datasets=datasets, concatenate=False)
    
    # Build representative SOAPs from training set (indices are relative to the training set)
    representative_idxs = np.loadtxt(f'{data_dir}/FPS_representatives.idxs', usecols=0, dtype=int)
    representative_soaps = np.vstack(soaps)[representative_idxs, :]
    
    # Build base kernels
    #KMM_base = build_kernel(representative_soaps, representative_soaps,
    #                   kernel=kernel_type, gamma=1.0)
    #KNM_base = build_kernel(soaps, representative_soaps,
    #                   kernel=kernel_type, gamma=1.0)
    
    # Loop over properties
    for pn in tqdm(property_names, desc='Property', leave=False):
        
        # Load the property values (just from the train set)
        Y = structure_properties[pn][train_idxs]
        
        # Initialize the list of errors
        errors_list = []
        
        # Loop over kernel parameters
        for g in tqdm(gamma, desc='Gamma', leave=False): 
            
            # Build kernel
            #KMM = KMM_base**g
            #KNM = KNM_base**g
            KMM = build_kernel(representative_soaps, representative_soaps,
                               kernel=kernel_type, gamma=g)
            KNM = build_kernel(soaps, representative_soaps,
                               kernel=kernel_type, gamma=g)
            
            # Loop over KMM regularization
            for s in tqdm(sigma, desc='Sigma', leave=False):
                
                # Loop over "eye" regularization
                for r in tqdm(reg, desc='Reg', leave=False):
                    
                    # Initialize containers for the errors of each fold
                    mae_train = np.zeros(k)
                    mae_validate = np.zeros(k)
                    rmse_train = np.zeros(k)
                    rmse_validate = np.zeros(k)
                    
                    # Loop over folds
                    for kk in tqdm(np.arange(0, k), desc='Iter', leave=False):
                        
                        # Set training and validation indices for the fold
                        idxs_train = cv_idxs.copy() # Different from train_idxs!
                        idxs_validate = idxs_train.pop(kk)
                        idxs_train = np.concatenate(idxs_train)
                        
                        # Center properties based on the training set of the fold
                        Yk = Y - np.mean(Y[idxs_train])        

                        # Set scaling
                        delta = np.var(Yk[idxs_train]) * KMM.shape[0] / np.trace(KMM)

                        # Initialize sparse KRR
                        skrr = SparseKRR(sigma=s, reg=r, reg_type=reg_type, rcond=None)
                        skrr.fit(delta*KNM[idxs_train, :], delta*KMM, delta*Yk[idxs_train])

                        # Compute predicted Y
                        Yp_train = skrr.transform(KNM[idxs_train, :])
                        Yp_validate = skrr.transform(KNM[idxs_validate, :])
                        
                        # Compute errors
                        mae_train[kk] = MAE(Yp_train, Yk[idxs_train])
                        mae_validate[kk] = MAE(Yp_validate, Yk[idxs_validate])
                        rmse_train[kk] = RMSE(Yp_train, Yk[idxs_train])
                        rmse_validate[kk] = RMSE(Yp_validate, Yk[idxs_validate])
                    
                    # Build the structured numpy array of errors
                    model = np.array([(g, s, r, mae_train, mae_validate, rmse_train, rmse_validate)],
                                     dtype=dt)    
                    errors_list.append(model)
        
        # Concatenate the list of structured numpy arrays of errors
        errors_list = np.concatenate(errors_list)
        
        # Stack the arrays in a writable form so we can save in plain text instead of npy/npz
        columns = []
        header = []
        for name in dt.names:
            column = errors_list[name]
            if column.ndim == 1:
                column = np.reshape(column, (-1, 1))
                header.append(name)
            else:
                n_cols = column.shape[1]
                header.append(f'{name}({n_cols})')
            columns.append(column)
        header = ' '.join(header)
        np.savetxt(f'../Processed_Data/Models/{cutoff}/{pn}_optimization.dat', 
                   np.hstack(columns), header=header)

In [12]:
# Extract optimal parameters

# Loop over cutoffs
for cutoff in cutoffs:
    
    # Loop over properties
    for pn in property_names:
        errors_list = np.loadtxt(f'../Processed_Data/DEEM_10k/Models/{cutoff}/{pn}_optimization.dat', dtype=dt)
        
        # Loop over error types
        for error in ('mae', 'rmse'):
        
            # Extract set of parameters corresponding to the minimum error
            idx = np.argmin(np.mean(errors_list[f'{error}_validate'], axis=1))

            # Print error and parameters
            print(f'-----Optimal Parameters for {cutoff} {pn} {error.upper()}-----')
            print('Avg. Error =', np.mean(errors_list[f'{error}_validate'][idx]))
            print('Gamma =', errors_list[idx]['gamma'])
            print('Sigma =', errors_list[idx]['sigma'])
            print('Regularization =', errors_list[idx]['reg'])
            print('')

            # Save optimal parameters
            opt_params = dict(kernel_type=kernel_type,
                              gamma=errors_list[idx]['gamma'],
                              sigma=errors_list[idx]['sigma'],
                              reg=errors_list[idx]['reg'])
            with open(f'../Processed_Data/DEEM_10k/Models/{cutoff}/{pn}_{error}_parameters.json', 'w') as f:
                json.dump(opt_params, f)

-----Optimal Parameters for 3.5 volumes MAE-----
Avg. Error = 2.5304993519468213
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 volumes RMSE-----
Avg. Error = 3.654212004317217
Gamma = 10.0
Sigma = 0.1
Regularization = 0.001

-----Optimal Parameters for 3.5 energies MAE-----
Avg. Error = 0.689774447323908
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 3.5 energies RMSE-----
Avg. Error = 0.9967709902316922
Gamma = 10.0
Sigma = 0.01
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes MAE-----
Avg. Error = 1.0099737440112349
Gamma = 10.0
Sigma = 1.0
Regularization = 0.0001

-----Optimal Parameters for 6.0 volumes RMSE-----
Avg. Error = 1.7406761608394494
Gamma = 1.0
Sigma = 0.01
Regularization = 0.01

-----Optimal Parameters for 6.0 energies MAE-----
Avg. Error = 0.47347945338816533
Gamma = 1.0
Sigma = 0.001
Regularization = 0.001

-----Optimal Parameters for 6.0 energies RMSE-----
Avg. Error = 0.73099440698692